<a id="section-1"></a>
# Section 1: Why State and Reasoning Must Be Separate (But Connected)

Let's start with the fundamental principle: **separation of concerns**.

---

## 1.1 State and Reasoning Are Distinct Concerns

### What Is State?

**State** is the **data** that an agent knows and remembers:

```
STATE = Data at rest

Examples:
• User query: "Find papers on transformers"
• Retrieved documents: [doc1, doc2, doc3]
• Selected papers: [paper_a, paper_b]
• Current step: 3
• Is complete: False
```

State is:
- **Persistent**: Survives across reasoning steps
- **Structured**: Typed fields with clear schemas
- **Observable**: Can be inspected at any point
- **Serializable**: Can be saved, loaded, replayed

### What Is Reasoning?

**Reasoning** is the **process** by which an agent decides what to do:

```
REASONING = Logic in motion

Examples:
• "I have the query, I should search for papers"
• "I found 10 papers, I should filter to top 3"
• "I have 3 papers, I can now synthesize"
• "My synthesis is complete, I should terminate"
```

Reasoning is:
- **Procedural**: A sequence of thoughts and decisions
- **Contextual**: Depends on current state
- **Transient** (if not recorded): Happens during execution
- **Actionable**: Produces state changes

---

## 1.2 Why They Must Be Separate

### Principle: Separation of Concerns

```
STATE:      "What do I know?"
REASONING:  "What should I do with what I know?"
```

These are fundamentally different questions.

### Why Separation Matters

| Aspect | If Mixed | If Separated |
|--------|----------|-------------|
| **Debugging** | Can't tell data from logic | Clear boundaries |
| **Testing** | Hard to test reasoning in isolation | Can unit test each |
| **Reusability** | Logic tied to specific state shape | Reasoning portable |
| **Validation** | Can't validate data vs decisions separately | Independent validation |
| **Observability** | Black box behavior | Transparent process |

### Example: The Wrong Way (Mixed)

```python
# BAD: Reasoning hidden in prompt
prompt = f"""
You are an agent. Your task is to find papers on {query}.

Think step by step:
1. Search for papers
2. Filter by relevance
3. Summarize findings

Now do it.
"""

response = llm.generate(prompt)
```

**Problems:**
- State (query) is buried in prompt string
- Reasoning (steps 1-3) is in prompt, not in state
- No way to verify steps actually happened
- Response is opaque text, not structured state

### Example: The Right Way (Separated)

```python
# GOOD: Clear separation
state = {
    "query": "Find papers on transformers",  # Data
    "papers": [],
    "current_step": 1
}

# Reasoning reads state, decides action
if state["current_step"] == 1:
    thought = "I should search for papers"
    action = search_papers(state["query"])
    
    # Reasoning writes results back to state
    state["papers"] = action.results
    state["reasoning_trace"].append({"thought": thought, "action": "search"})
    state["current_step"] = 2
```

**Benefits:**
- State is structured data
- Reasoning is explicit code
- Can verify each step
- Can inspect state at any point

---

## 1.3 Why Reasoning Should NOT Live Inside Prompts

Putting reasoning in prompts is one of the most common anti-patterns.

### The Prompt-Based "Reasoning" Trap

```
Prompt: "Think step by step and solve this problem"
         ↓
    [LLM generates text that LOOKS like reasoning]
         ↓
    But it's just text, not actual steps
```

### What's Wrong With This?

| Problem | Why It Fails |
|---------|-------------|
| **No verification** | Can't check if steps actually executed |
| **No branching** | Can't take different paths based on intermediate results |
| **No tool use** | Can't call real tools between steps |
| **No state update** | No persistent memory of what happened |
| **Hallucination risk** | LLM can claim to have done things it didn't |

### The Correct Approach: Externalized Reasoning

```
Reasoning as CODE, not PROMPTS:

1. Read current state
2. Decide what to do (reasoning logic)
3. Execute action (tool call, API, etc.)
4. Record decision in state
5. Write results to state
6. Repeat
```

Now reasoning is:
- ✅ Verifiable (in code)
- ✅ Traceable (recorded in state)
- ✅ Testable (unit tests)
- ✅ Observable (state inspection)

---

## 1.4 Why State Without Reasoning Is Inert

State alone cannot do anything.

### State = Passive Data

```python
state = {
    "query": "Find papers on transformers",
    "papers": [],
    "current_step": 1,
    "is_complete": False
}

# This state just sits there. Nothing happens.
```

**State has no agency.** It doesn't:
- Decide what to do next
- Execute actions
- Update itself
- Reach conclusions

### Reasoning = Active Logic

Reasoning is what **brings state to life**:

```python
# Reasoning looks at state and decides
if not state["papers"]:
    # "I need to search!"
    papers = search(state["query"])
    state["papers"] = papers
    state["current_step"] = 2
```

Without reasoning:
- State never changes
- No progress toward goal
- Agent is frozen

---

## 1.5 Why Reasoning Without State Is Brittle

Reasoning without state has no memory or context.

### Stateless Reasoning = Amnesia

```python
# BAD: Reasoning with no persistent state
def step_1():
    papers = search("transformers")
    # Papers are lost after this function returns

def step_2():
    # How do I access papers from step_1?
    # I can't! No state was saved.
```

**Problems:**
- Each step starts from scratch
- Can't build on previous work
- Must re-retrieve data every time
- No way to trace history

### Reasoning WITH State = Memory

```python
# GOOD: Reasoning reads/writes state
def step_1(state):
    papers = search(state["query"])
    state["papers"] = papers  # Saved!
    return state

def step_2(state):
    # Papers are available from state
    filtered = filter_papers(state["papers"])
    state["filtered_papers"] = filtered
    return state
```

**Benefits:**
- Work persists across steps
- Can build incrementally
- Full history available
- Debuggable progression

---

## 1.6 The Connection: Why They Need Each Other

```
┌─────────────────────────────────────────────────────────┐
│              STATE ↔ REASONING                          │
└─────────────────────────────────────────────────────────┘

STATE provides:                    REASONING provides:
• Context for decisions            • Logic to update state
• Memory of what happened          • Decisions on what to do next
• Data for reasoning to use        • Actions that change state
• Persistence across steps         • Progress toward goals

        STATE ──→ [REASONING] ──→ STATE'
          ▲                          │
          │                          │
          └──────────────────────────┘
              Feedback loop
```

### The Symbiotic Relationship

1. **Reasoning reads state** to understand context
2. **Reasoning decides** what action to take
3. **Reasoning executes** the action
4. **Reasoning writes** results back to state
5. **State grows** with new information
6. **Next reasoning step** has richer context
7. **Loop repeats** until goal achieved

This is the heartbeat of an agent.

---

## 🎯 Key Takeaway: Section 1

> **State and reasoning are distinct but inseparable concerns. State is passive data; reasoning is active logic. State without reasoning is inert. Reasoning without state is brittle. Together, in a feedback loop, they create intelligent agent behavior. Never mix them—always keep clean boundaries.**

---

<a id="section-2"></a>
# Section 2: The Read → Reason → Write Loop

This is the **fundamental pattern** that defines all agent behavior.

---

## 2.1 The Core Loop

Every agent step follows this pattern:

```
1. READ from state
   ↓
2. REASON about what to do
   ↓
3. WRITE results back to state
   ↓
[Repeat until goal achieved]
```

This is not optional—it's **the architecture** of an agent.

---

## 2.2 Breaking Down Each Phase

### Phase 1: READ from State

**What happens:**
- Agent examines current state
- Retrieves relevant data fields
- Understands current context

**Example:**
```
Read:
• What is my goal? → "Find papers on transformers"
• What have I done so far? → "Nothing yet, step 1"
• What data do I have? → "No papers retrieved yet"
```

**Key principle:** Reasoning ALWAYS starts by reading state.

### Phase 2: REASON About What to Do

**What happens:**
- Agent analyzes what it read
- Decides on next action
- Formulates thought/justification

**Example:**
```
Reasoning:
• "I see my goal is to find papers"
• "I haven't searched yet"
• "Therefore, I should search the database"
• Decision: Execute search action
```

**Key principle:** Reasoning produces a decision and action.

### Phase 3: WRITE Results to State

**What happens:**
- Action is executed
- Results are obtained
- State is updated with:
  - New data (from action)
  - Reasoning trace (thought + decision)
  - Progress markers (step number, etc.)

**Example:**
```
Write:
• papers ← [found 10 papers]
• reasoning_trace ← [{thought: "I should search", action: "search"}]
• current_step ← 2
```

**Key principle:** State always grows after each iteration.

---

## 2.3 Conceptual Diagram

```
┌──────────────────────────────────────────────────────────────┐
│           READ → REASON → WRITE LOOP                         │
└──────────────────────────────────────────────────────────────┘

        ┌─────────────────────────────────────┐
        │         AGENT STATE                 │
        │                                     │
        │  • goal: "Find papers"              │
        │  • papers: []                       │
        │  • current_step: 1                  │
        │  • reasoning_trace: []              │
        └──────────────┬──────────────────────┘
                       │
                       │ (1) READ
                       │ "What do I know?"
                       │
                       ▼
        ┌──────────────────────────────────────┐
        │      REASONING ENGINE                │
        │                                      │
        │  Input:                              │
        │  • goal = "Find papers"              │
        │  • papers = []                       │
        │  • current_step = 1                  │
        │                                      │
        │  (2) REASON                          │
        │  "What should I do?"                 │
        │                                      │
        │  Logic:                              │
        │  • I have a goal but no papers       │
        │  • I'm on step 1                     │
        │  • → Decision: SEARCH for papers     │
        │                                      │
        │  Action:                             │
        │  • Execute search("transformers")    │
        │  • Result: [10 papers found]         │
        │                                      │
        └──────────────┬───────────────────────┘
                       │
                       │ (3) WRITE
                       │ "Update state with results"
                       │
                       ▼
        ┌─────────────────────────────────────┐
        │      UPDATED STATE                  │
        │                                     │
        │  • goal: "Find papers"              │
        │  • papers: [10 papers] ← NEW!       │
        │  • current_step: 2 ← UPDATED!       │
        │  • reasoning_trace: [{thought, action}] ← ADDED!
        └──────────────┬──────────────────────┘
                       │
                       │ Goal achieved?
                       │
                ┌──────┴──────┐
                │             │
               NO            YES
                │             │
                │             ▼
                │     ┌──────────────┐
                │     │  TERMINATE   │
                │     │ Return Result│
                │     └──────────────┘
                │
                └──────► [LOOP BACK TO READ]
                         State is richer now
```

---

## 2.4 Why This Loop Defines Agent Behavior

### 1. It Creates Incremental Progress

Each iteration:
- Builds on previous work
- Adds new information
- Moves closer to goal

```
Iteration 1: Search → Find 10 papers
Iteration 2: Filter → Select top 3
Iteration 3: Synthesize → Generate summary
Iteration 4: Check complete → Terminate
```

### 2. It Enables Adaptation

Reasoning can branch based on what it reads:

```
If papers < 5:
    → Search again with broader query
Else if papers > 100:
    → Filter more aggressively
Else:
    → Proceed to synthesis
```

### 3. It Provides Observability

At any point, you can inspect:
- Current state
- Reasoning trace
- Progress toward goal

### 4. It Ensures Grounding

Every decision is based on **actual state**, not assumptions:

```
✅ GOOD: "I have 10 papers in state, I'll filter them"
❌ BAD:  "I assume I have papers, I'll pretend to filter"
```

---

## 2.5 Why Every Agent Step Should Follow This Pattern

### Consistency = Predictability

If every step follows Read → Reason → Write:
- Easier to understand
- Easier to debug
- Easier to test
- Easier to maintain

### Example: Inconsistent Pattern (BAD)

```python
# Step 1: Follows pattern
state = read_state()
action = reason(state)
write_state(action.result)

# Step 2: Doesn't follow pattern (BAD!)
result = magic_llm_call("Just do it")  # No state read
print(result)  # No state write

# Step 3: Now what? State is inconsistent!
```

**Problem:** Step 2 broke the pattern. Now:
- State doesn't reflect what happened in Step 2
- Step 3 has incomplete context
- Debugging is impossible

### Example: Consistent Pattern (GOOD)

```python
# Every step follows the same pattern
for step in range(max_steps):
    # 1. READ
    state = read_current_state()
    
    # 2. REASON
    decision = reasoning_engine(state)
    
    # 3. WRITE
    state = write_results(state, decision)
    
    if state.is_complete:
        break
```

**Benefits:**
- Uniform structure
- Clear progression
- Easy to trace
- Reliable behavior

---

## 2.6 The Loop in Action: Example Trace

Let's trace a simple agent through 3 iterations:

### Initial State
```
state = {
    "goal": "Summarize papers on transformers",
    "papers": [],
    "summary": None,
    "current_step": 1
}
```

### Iteration 1
```
READ:    goal="Summarize papers", papers=[], step=1
REASON:  "I need papers first" → Action: search
WRITE:   papers=[10 papers], step=2
```

### Iteration 2
```
READ:    papers=[10 papers], step=2
REASON:  "Too many, filter to top 3" → Action: filter
WRITE:   papers=[3 papers], step=3
```

### Iteration 3
```
READ:    papers=[3 papers], step=3
REASON:  "Now I can summarize" → Action: synthesize
WRITE:   summary="...", is_complete=True
```

### Final State
```
state = {
    "goal": "Summarize papers on transformers",
    "papers": [3 papers],
    "summary": "Transformers use self-attention...",
    "current_step": 4,
    "is_complete": True
}
```

Notice how state **evolved** through each iteration.

---

## 🎯 Key Takeaway: Section 2

> **The Read → Reason → Write loop is the heartbeat of an agent. Every step must follow this pattern: read current state, reason about what to do, write results back. This creates incremental progress, enables adaptation, provides observability, and ensures grounding. Consistency in this pattern is critical for maintainability.**

---

<a id="section-3"></a>
# Section 3: What Reasoning Reads vs What It Writes

Not all state fields are equal. Some are inputs, some are outputs, and some should never be modified.

---

## 3.1 Three Categories of State Fields

### Category 1: READ-ONLY Fields

**Definition:** Fields that reasoning reads but NEVER modifies.

**Examples:**
- User query
- Initial configuration
- System parameters
- Immutable inputs

**Why read-only?**
- Preserve original intent
- Prevent accidental overwrites
- Enable replay/debugging

```python
# Example
state = {
    "user_query": "Find papers on transformers",  # READ-ONLY
    "max_papers": 10,                             # READ-ONLY
    # Reasoning should NEVER change these
}
```

### Category 2: READ-WRITE Fields

**Definition:** Fields that reasoning both reads and writes.

**Examples:**
- Retrieved data
- Intermediate results
- Processing state
- Reasoning trace

**Why read-write?**
- Accumulate information
- Track progress
- Build incrementally

```python
# Example
state = {
    "papers": [],              # READ-WRITE: starts empty, grows
    "filtered_papers": [],     # READ-WRITE: derived from papers
    "current_step": 1,         # READ-WRITE: increments each iteration
    "reasoning_trace": [],     # READ-WRITE: appends each decision
}
```

### Category 3: WRITE-ONLY Fields

**Definition:** Fields that reasoning writes but typically doesn't read (until later).

**Examples:**
- Final output
- Completion flag
- Error messages

**Why write-only?**
- Mark completion
- Store final result
- Signal termination

```python
# Example
state = {
    "final_summary": None,     # WRITE-ONLY: set once at end
    "is_complete": False,      # WRITE-ONLY: set to True when done
    "error": None,             # WRITE-ONLY: set if something fails
}
```

---

## 3.2 What Belongs in State BEFORE Reasoning

State should be initialized with:

### 1. The Goal/Objective
```python
state["goal"] = "Summarize recent papers on transformers"
```
**Why:** Reasoning needs to know what to achieve.

### 2. Initial Inputs
```python
state["user_query"] = "transformers"
state["year_filter"] = 2020
```
**Why:** Parameters for the task.

### 3. Empty Containers for Results
```python
state["papers"] = []
state["reasoning_trace"] = []
state["final_summary"] = None
```
**Why:** Reasoning will fill these.

### 4. Control Flags
```python
state["current_step"] = 1
state["is_complete"] = False
state["max_iterations"] = 10
```
**Why:** Track progress and limits.

---

## 3.3 What Reasoning Is Allowed to Modify

Reasoning can modify:

### ✅ ALLOWED: Data Fields
```python
state["papers"] = search_results  # Adding retrieved data
state["filtered_papers"] = top_3  # Derived data
```

### ✅ ALLOWED: Progress Markers
```python
state["current_step"] += 1  # Incrementing step counter
state["is_complete"] = True  # Marking done
```

### ✅ ALLOWED: Reasoning Trace
```python
state["reasoning_trace"].append({
    "thought": "I should search",
    "action": "search",
    "observation": "Found 10 papers"
})
```

### ✅ ALLOWED: Final Outputs
```python
state["final_summary"] = generated_summary
```

---

## 3.4 What Should NEVER Be Overwritten

### ❌ FORBIDDEN: Initial Inputs
```python
# BAD: Don't modify the original query
state["user_query"] = "something else"  # NEVER DO THIS
```
**Why:** Loses original context, breaks replay.

### ❌ FORBIDDEN: Configuration Parameters
```python
# BAD: Don't change system limits mid-execution
state["max_iterations"] = 100  # NEVER DO THIS
```
**Why:** Violates constraints, unpredictable behavior.

### ❌ FORBIDDEN: Historical Data Without Appending
```python
# BAD: Don't overwrite reasoning history
state["reasoning_trace"] = [new_step]  # NEVER DO THIS

# GOOD: Append to history
state["reasoning_trace"].append(new_step)  # DO THIS
```
**Why:** Loses audit trail.

---

## 3.5 Read-Only vs Write-Only Checklist

### ☑️ Read-Only Field Checklist

A field should be read-only if:

- ☐ It represents the original user input
- ☐ It's a system configuration parameter
- ☐ Modifying it would change the task definition
- ☐ It's needed for replay or debugging
- ☐ It's set once at initialization

**Examples:**
- `user_query`
- `max_papers`
- `timeout_seconds`
- `api_endpoint`

### ☑️ Write-Only Field Checklist

A field should be write-only if:

- ☐ It's the final output of the agent
- ☐ It's a completion/termination flag
- ☐ It's written once and never read during reasoning
- ☐ It signals the end of execution

**Examples:**
- `final_summary`
- `is_complete`
- `final_result`
- `error_message`

### ☑️ Read-Write Field Checklist

A field should be read-write if:

- ☐ It accumulates data over multiple steps
- ☐ It's an intermediate result
- ☐ It's both input to and output from reasoning
- ☐ It evolves throughout execution

**Examples:**
- `papers` (empty → filled)
- `current_step` (1 → 2 → 3)
- `reasoning_trace` (appends each step)
- `partial_results` (builds incrementally)

---

## 3.6 Conceptual State Schema

Here's a template for organizing state fields:

```
┌────────────────────────────────────────────────────────────┐
│                   AGENT STATE SCHEMA                       │
└────────────────────────────────────────────────────────────┘

READ-ONLY SECTION (Inputs & Config)
────────────────────────────────────────────────────────────
  user_query: str          # What the user asked
  max_papers: int          # System limit
  year_filter: int         # Constraint
  api_key: str             # Configuration

READ-WRITE SECTION (Working Memory)
────────────────────────────────────────────────────────────
  papers: List[Paper]              # Retrieved data
  filtered_papers: List[Paper]     # Processed data
  current_step: int                # Progress tracker
  reasoning_trace: List[Step]      # Decision history
  intermediate_results: Dict       # Scratch space

WRITE-ONLY SECTION (Outputs)
────────────────────────────────────────────────────────────
  final_summary: Optional[str]     # Final output
  is_complete: bool                # Termination flag
  error: Optional[str]             # Error state
  total_steps_taken: int           # Metrics

METADATA SECTION (System)
────────────────────────────────────────────────────────────
  created_at: timestamp
  last_updated: timestamp
  agent_version: str
```

---

## 3.7 Design Guidelines

### When Designing State Fields

1. **Be explicit about mutability**
   - Document which fields are read-only
   - Enforce with code (e.g., frozen dataclasses)

2. **Separate concerns**
   - Inputs in one section
   - Working data in another
   - Outputs in a third

3. **Never silently overwrite**
   - Always append to history fields
   - Log when modifying important fields

4. **Use naming conventions**
   - `initial_*` for read-only inputs
   - `current_*` for read-write working data
   - `final_*` for write-only outputs

---

## 3.8 Example: Well-Designed State

```python
# Conceptual example (no code yet, just structure)

state = {
    # READ-ONLY: Initial inputs (never modified)
    "initial_query": "Find papers on transformers",
    "initial_year_filter": 2020,
    "initial_max_papers": 10,
    
    # READ-WRITE: Working memory (evolves)
    "current_papers": [],  # Grows as we search
    "current_step": 1,     # Increments each iteration
    "reasoning_trace": [], # Appends each decision
    
    # WRITE-ONLY: Final outputs (set at end)
    "final_summary": None,
    "is_complete": False,
}
```

**Benefits of this structure:**
- ✅ Clear boundaries
- ✅ Self-documenting
- ✅ Easy to validate
- ✅ Hard to misuse

---

## 🎯 Key Takeaway: Section 3

> **State fields fall into three categories: read-only (inputs that never change), read-write (working memory that evolves), and write-only (final outputs). Understanding these boundaries is critical for clean state-reasoning interaction. Always preserve inputs, append to history, and document mutability.**

---

# Summary: Sections 1-3

| Section | Key Concept |
|---------|-------------|
| **Section 1** | State and reasoning are separate but connected concerns |
| **Section 2** | The Read → Reason → Write loop defines all agent behavior |
| **Section 3** | State fields have clear read-only vs read-write vs write-only boundaries |

---

## Next Sections Preview

In upcoming sections, we will:

- **Section 4:** Implement the Read-Reason-Write pattern in code
- **Section 5:** Trace state evolution through multiple reasoning steps
- **Section 6:** Enforce read-only vs write-only field constraints
- **Section 7:** Examine anti-patterns and how to avoid them
- **Section 8:** Debug state-reasoning interactions
- **Section 9:** Coordinate state in multi-agent systems
- **Section 10:** Production patterns and best practices

---

<a id="section-4"></a>
# Section 4: Single-Agent State ↔ Reasoning Loop

Now let's implement the **Read → Reason → Write** pattern with actual code.

---

## 4.1 Defining the State Schema

First, we'll define a state structure for a simple research agent:


In [ ]:
from typing import TypedDict, Optional, List
from dataclasses import dataclass, field
from copy import deepcopy
import json

# Define state schema with clear read/write boundaries
@dataclass
class ResearchAgentState:
    """
    State for a simple research agent.
    
    READ-ONLY fields (set at initialization):
    - initial_query: The user's original query
    - max_papers: Maximum number of papers to retrieve
    
    READ-WRITE fields (evolve during execution):
    - papers: Retrieved papers
    - filtered_papers: Selected papers
    - current_step: Current iteration number
    - reasoning_trace: History of decisions
    
    WRITE-ONLY fields (set at completion):
    - final_summary: Final output
    - is_complete: Termination flag
    """
    # READ-ONLY: Inputs (never modified after initialization)
    initial_query: str
    max_papers: int
    
    # READ-WRITE: Working memory (evolves)
    papers: List[dict] = field(default_factory=list)
    filtered_papers: List[dict] = field(default_factory=list)
    current_step: int = 1
    reasoning_trace: List[dict] = field(default_factory=list)
    
    # WRITE-ONLY: Final outputs (set once)
    final_summary: Optional[str] = None
    is_complete: bool = False
    
    def to_dict(self):
        """Convert state to dictionary for display."""
        return {
            "READ-ONLY": {
                "initial_query": self.initial_query,
                "max_papers": self.max_papers,
            },
            "READ-WRITE": {
                "papers": self.papers,
                "filtered_papers": self.filtered_papers,
                "current_step": self.current_step,
                "reasoning_trace": self.reasoning_trace,
            },
            "WRITE-ONLY": {
                "final_summary": self.final_summary,
                "is_complete": self.is_complete,
            }
        }

# Initialize state
state = ResearchAgentState(
    initial_query="Find papers on transformer architectures",
    max_papers=5
)

print("Initial State:")
print(json.dumps(state.to_dict(), indent=2))


## 4.2 Mock Tools for Agent Actions

We'll create simple mock tools that the agent can use:


In [ ]:
# Mock tools that simulate agent actions
def search_papers(query: str, max_results: int) -> List[dict]:
    """Mock tool: Search for papers."""
    # Simulated search results
    return [
        {"id": 1, "title": "Attention Is All You Need", "citations": 50000, "year": 2017},
        {"id": 2, "title": "BERT: Pre-training of Deep Bidirectional Transformers", "citations": 30000, "year": 2019},
        {"id": 3, "title": "GPT-3: Language Models are Few-Shot Learners", "citations": 20000, "year": 2020},
        {"id": 4, "title": "Transformer-XL: Attentive Language Models", "citations": 5000, "year": 2019},
        {"id": 5, "title": "Reformer: The Efficient Transformer", "citations": 3000, "year": 2020},
    ][:max_results]

def filter_papers(papers: List[dict], min_citations: int) -> List[dict]:
    """Mock tool: Filter papers by citation count."""
    return [p for p in papers if p["citations"] >= min_citations]

def synthesize_summary(papers: List[dict]) -> str:
    """Mock tool: Generate summary from papers."""
    titles = [p["title"] for p in papers]
    return f"Summary of {len(papers)} papers: {', '.join(titles)}"

print("Mock tools defined successfully")


## 4.3 Implementing the Read → Reason → Write Loop

Now let's implement a single reasoning step that follows the pattern:


In [ ]:
def reasoning_step(state: ResearchAgentState) -> ResearchAgentState:
    """
    Execute one Read → Reason → Write cycle.
    
    Returns updated state with reasoning trace.
    """
    print(f"\n{'='*60}")
    print(f"STEP {state.current_step}: Starting reasoning cycle")
    print(f"{'='*60}\n")
    
    # ===== PHASE 1: READ from state =====
    print("📖 PHASE 1: READ from state")
    print("-" * 40)
    
    # Read current state
    query = state.initial_query
    max_papers = state.max_papers
    papers = state.papers
    filtered_papers = state.filtered_papers
    step = state.current_step
    
    print(f"  Query: {query}")
    print(f"  Max papers: {max_papers}")
    print(f"  Papers retrieved: {len(papers)}")
    print(f"  Papers filtered: {len(filtered_papers)}")
    print(f"  Current step: {step}")
    
    # ===== PHASE 2: REASON about what to do =====
    print("\n🧠 PHASE 2: REASON about next action")
    print("-" * 40)
    
    thought = None
    action = None
    observation = None
    
    # Decision logic based on state
    if step == 1 and len(papers) == 0:
        thought = "I need to search for papers on the query"
        action = "search"
        print(f"  Thought: {thought}")
        print(f"  Decision: Execute {action} action")
        
        # Execute action
        result = search_papers(query, max_papers)
        observation = f"Found {len(result)} papers"
        
    elif step == 2 and len(filtered_papers) == 0:
        thought = "I should filter papers to keep only high-impact ones"
        action = "filter"
        print(f"  Thought: {thought}")
        print(f"  Decision: Execute {action} action")
        
        # Execute action
        result = filter_papers(papers, min_citations=10000)
        observation = f"Filtered to {len(result)} high-impact papers"
        
    elif step == 3 and state.final_summary is None:
        thought = "I have filtered papers, now I can synthesize a summary"
        action = "synthesize"
        print(f"  Thought: {thought}")
        print(f"  Decision: Execute {action} action")
        
        # Execute action
        result = synthesize_summary(filtered_papers)
        observation = f"Generated summary of {len(filtered_papers)} papers"
        
    else:
        thought = "Task is complete, nothing more to do"
        action = "terminate"
        result = None
        observation = "All steps completed"
        print(f"  Thought: {thought}")
        print(f"  Decision: {action}")
    
    print(f"  Observation: {observation}")
    
    # ===== PHASE 3: WRITE results back to state =====
    print("\n✍️  PHASE 3: WRITE results to state")
    print("-" * 40)
    
    # Update state based on action
    if action == "search":
        state.papers = result
        print(f"  Updated 'papers': {len(result)} papers added")
        
    elif action == "filter":
        state.filtered_papers = result
        print(f"  Updated 'filtered_papers': {len(result)} papers selected")
        
    elif action == "synthesize":
        state.final_summary = result
        state.is_complete = True
        print(f"  Updated 'final_summary': Summary generated")
        print(f"  Updated 'is_complete': True")
        
    elif action == "terminate":
        state.is_complete = True
        print(f"  Updated 'is_complete': True")
    
    # Always append to reasoning trace
    state.reasoning_trace.append({
        "step": step,
        "thought": thought,
        "action": action,
        "observation": observation
    })
    print(f"  Appended to 'reasoning_trace': Step {step} recorded")
    
    # Always increment step counter
    state.current_step += 1
    print(f"  Updated 'current_step': {state.current_step}")
    
    return state

print("Reasoning step function defined")


## 4.4 Running the Agent: Complete Execution

Now let's run the agent through all its steps and observe state evolution:


In [ ]:
# Reset state for clean execution
state = ResearchAgentState(
    initial_query="Find papers on transformer architectures",
    max_papers=5
)

print("🚀 Starting Agent Execution")
print("="*60)

# Run agent loop
max_iterations = 5
for iteration in range(max_iterations):
    # Execute one reasoning step
    state = reasoning_step(state)
    
    # Print current state after step
    print(f"\n📊 State after step {iteration + 1}:")
    print(json.dumps(state.to_dict(), indent=2))
    
    # Check if complete
    if state.is_complete:
        print(f"\n✅ Agent completed in {iteration + 1} steps")
        break
else:
    print(f"\n⚠️  Agent did not complete within {max_iterations} iterations")

print("\n" + "="*60)
print("🏁 Final Agent State:")
print("="*60)
print(json.dumps(state.to_dict(), indent=2))


## 4.5 Key Observations

Notice how the agent demonstrates the Read → Reason → Write pattern:

1. **State Before Each Step**: Shows what the agent knows
2. **Reasoning Phase**: Agent decides based on current state
3. **State After Each Step**: Shows how state evolved

**Critical Points:**

- ✅ **READ-ONLY fields never changed**: `initial_query` and `max_papers` stayed constant
- ✅ **READ-WRITE fields evolved**: `papers`, `filtered_papers`, `reasoning_trace` grew
- ✅ **WRITE-ONLY fields set once**: `final_summary` and `is_complete` set at end
- ✅ **Reasoning always recorded**: Every decision is in `reasoning_trace`
- ✅ **State drives decisions**: Each step's action depends on current state

---

<a id="section-5"></a>
# Section 5: Validation of State–Reasoning Interaction

Validation ensures that state-reasoning interactions follow the rules.

---

## 5.1 Why Validation Is Critical

**Without validation:**
- Reasoning might modify read-only fields
- Required outputs might not be written
- State might become inconsistent
- Bugs are hard to detect

**With validation:**
- ✅ Enforce architectural constraints
- ✅ Catch bugs early
- ✅ Ensure predictable behavior
- ✅ Enable safe refactoring

---

## 5.2 Validation Rules

We'll validate three key properties:

1. **Read-only fields are never modified**
2. **Expected outputs are written**
3. **Reasoning trace is properly maintained**


In [ ]:
def validate_state_interaction(
    state_before: ResearchAgentState,
    state_after: ResearchAgentState,
    step_number: int
) -> dict:
    """
    Validate that state-reasoning interaction follows the rules.
    
    Returns:
        dict with validation results
    """
    errors = []
    warnings = []
    
    # Rule 1: Read-only fields must NOT change
    if state_before.initial_query != state_after.initial_query:
        errors.append(f"Step {step_number}: READ-ONLY field 'initial_query' was modified")
    
    if state_before.max_papers != state_after.max_papers:
        errors.append(f"Step {step_number}: READ-ONLY field 'max_papers' was modified")
    
    # Rule 2: Reasoning trace must grow (append-only)
    if len(state_after.reasoning_trace) != len(state_before.reasoning_trace) + 1:
        errors.append(
            f"Step {step_number}: reasoning_trace should grow by 1, "
            f"but grew by {len(state_after.reasoning_trace) - len(state_before.reasoning_trace)}"
        )
    
    # Rule 3: Step counter must increment
    if state_after.current_step != state_before.current_step + 1:
        errors.append(
            f"Step {step_number}: current_step should increment by 1, "
            f"but changed from {state_before.current_step} to {state_after.current_step}"
        )
    
    # Rule 4: If papers are retrieved, they should be non-empty
    if len(state_after.papers) > 0 and len(state_after.papers) == len(state_before.papers):
        # Papers didn't change this step, that's ok
        pass
    elif len(state_after.papers) > 0 and len(state_before.papers) == 0:
        # Good: papers were retrieved
        pass
    
    # Rule 5: Reasoning trace should reference actual state
    if len(state_after.reasoning_trace) > 0:
        last_trace = state_after.reasoning_trace[-1]
        if last_trace["step"] != step_number:
            warnings.append(
                f"Step {step_number}: reasoning_trace step number mismatch "
                f"(expected {step_number}, got {last_trace['step']})"
            )
    
    # Rule 6: is_complete should only be set to True, never back to False
    if state_before.is_complete and not state_after.is_complete:
        errors.append(f"Step {step_number}: is_complete was set back to False")
    
    return {
        "valid": len(errors) == 0,
        "errors": errors,
        "warnings": warnings
    }

print("Validation function defined")


## 5.3 Demonstration: Valid Interaction

Let's run the agent with validation:


In [ ]:
# Reset state
state = ResearchAgentState(
    initial_query="Find papers on transformer architectures",
    max_papers=5
)

print("✅ Running agent with validation\n")

for iteration in range(5):
    # Save state before
    state_before = deepcopy(state)
    
    # Execute reasoning step
    state = reasoning_step(state)
    
    # Validate
    validation = validate_state_interaction(state_before, state, iteration + 1)
    
    print(f"\n🔍 Validation Results for Step {iteration + 1}:")
    if validation["valid"]:
        print("  ✅ All validation checks passed")
    else:
        print("  ❌ Validation FAILED")
        for error in validation["errors"]:
            print(f"     - {error}")
    
    if validation["warnings"]:
        for warning in validation["warnings"]:
            print(f"     ⚠️  {warning}")
    
    if state.is_complete:
        break

print("\n✅ All steps validated successfully!")


## 5.4 Demonstration: BROKEN Interaction

Now let's demonstrate what happens when reasoning violates the rules:


In [ ]:
def broken_reasoning_step(state: ResearchAgentState) -> ResearchAgentState:
    """
    BAD EXAMPLE: This reasoning function violates state-reasoning rules.
    """
    print(f"\n❌ BROKEN STEP {state.current_step}")
    print("-" * 40)
    
    # VIOLATION 1: Modifying read-only field
    print("❌ VIOLATION 1: Modifying initial_query (read-only)")
    state.initial_query = "MODIFIED QUERY"  # BAD!
    
    # VIOLATION 2: Overwriting reasoning trace instead of appending
    print("❌ VIOLATION 2: Overwriting reasoning_trace (should append)")
    state.reasoning_trace = [{"step": state.current_step, "action": "bad"}]  # BAD!
    
    # VIOLATION 3: Not incrementing step counter
    print("❌ VIOLATION 3: Not incrementing current_step")
    # state.current_step += 1  # Forgot to increment!
    
    return state

# Test broken reasoning
print("🚨 Testing BROKEN reasoning step\n")

state = ResearchAgentState(
    initial_query="Original query",
    max_papers=5
)

state_before = deepcopy(state)
state = broken_reasoning_step(state)

# Validate
validation = validate_state_interaction(state_before, state, 1)

print(f"\n🔍 Validation Results:")
print(f"  Valid: {validation['valid']}")

if validation["errors"]:
    print("\n  ❌ ERRORS DETECTED:")
    for error in validation["errors"]:
        print(f"     - {error}")

print("\n📝 This demonstrates why validation is critical!")


## 5.5 Corrected Interaction

Now let's fix the violations:


In [ ]:
def corrected_reasoning_step(state: ResearchAgentState) -> ResearchAgentState:
    """
    GOOD EXAMPLE: This reasoning function follows all rules.
    """
    print(f"\n✅ CORRECTED STEP {state.current_step}")
    print("-" * 40)
    
    # CORRECT 1: Read from initial_query, never modify it
    query = state.initial_query  # Read only
    print(f"✅ READ from initial_query: '{query}'")
    
    # CORRECT 2: Append to reasoning trace, never overwrite
    state.reasoning_trace.append({
        "step": state.current_step,
        "action": "example_action",
        "thought": "This is the correct way"
    })
    print(f"✅ APPEND to reasoning_trace (length: {len(state.reasoning_trace)})")
    
    # CORRECT 3: Always increment step counter
    state.current_step += 1
    print(f"✅ INCREMENT current_step to {state.current_step}")
    
    return state

# Test corrected reasoning
print("✅ Testing CORRECTED reasoning step\n")

state = ResearchAgentState(
    initial_query="Original query",
    max_papers=5
)

state_before = deepcopy(state)
state = corrected_reasoning_step(state)

# Validate
validation = validate_state_interaction(state_before, state, 1)

print(f"\n🔍 Validation Results:")
print(f"  Valid: {validation['valid']}")

if validation["valid"]:
    print("  ✅ All rules followed correctly!")
else:
    print("  ❌ Unexpected errors:")
    for error in validation["errors"]:
        print(f"     - {error}")


## 5.6 Why Validation Is Critical

**Lessons from this section:**

1. **Validation catches violations early** before they cause downstream bugs
2. **Clear rules** make it easy to write correct reasoning logic
3. **Automated checks** prevent human error
4. **Broken interactions** create inconsistent state that's hard to debug

**Best Practice:**
- Always validate state transitions in development
- Use type hints and frozen fields where possible
- Document read-only vs read-write fields clearly
- Test both valid and invalid interactions

---

<a id="section-6"></a>
# Section 6: Interaction in a RAG Agent

Let's see how state-reasoning interaction works in a **Retrieval-Augmented Generation (RAG)** agent.

---

## 6.1 RAG Agent State Schema

A RAG agent needs to track:
- Query
- Retrieved documents
- Selected documents
- Reasoning decisions about when to retrieve, select, and synthesize


In [ ]:
@dataclass
class RAGAgentState:
    """
    State for a RAG agent that retrieves and synthesizes information.
    
    Tracks reasoning decisions explicitly.
    """
    # READ-ONLY: Initial inputs
    query: str
    max_documents: int
    
    # READ-WRITE: Retrieved and processed data
    retrieved_documents: List[dict] = field(default_factory=list)
    selected_documents: List[dict] = field(default_factory=list)
    
    # READ-WRITE: Reasoning decisions
    reasoning_trace: List[dict] = field(default_factory=list)
    current_phase: str = "analyze_query"  # analyze_query → retrieve → select → synthesize
    
    # WRITE-ONLY: Final output
    final_answer: Optional[str] = None
    is_complete: bool = False
    
    def to_dict(self):
        return {
            "READ-ONLY": {
                "query": self.query,
                "max_documents": self.max_documents,
            },
            "READ-WRITE": {
                "retrieved_documents": len(self.retrieved_documents),
                "selected_documents": len(self.selected_documents),
                "current_phase": self.current_phase,
                "reasoning_trace": self.reasoning_trace,
            },
            "WRITE-ONLY": {
                "final_answer": self.final_answer,
                "is_complete": self.is_complete,
            }
        }

print("RAG Agent State schema defined")


## 6.2 Mock RAG Tools


In [ ]:
def retrieve_documents(query: str, max_docs: int) -> List[dict]:
    """Mock: Retrieve documents from a vector database."""
    return [
        {"id": 1, "content": "Transformers use self-attention mechanisms...", "relevance": 0.95},
        {"id": 2, "content": "BERT is a transformer-based model for NLP...", "relevance": 0.90},
        {"id": 3, "content": "GPT uses decoder-only transformer architecture...", "relevance": 0.88},
        {"id": 4, "content": "Attention mechanisms allow models to focus...", "relevance": 0.75},
        {"id": 5, "content": "Neural networks process information...", "relevance": 0.60},
    ][:max_docs]

def select_relevant_documents(documents: List[dict], threshold: float) -> List[dict]:
    """Mock: Select documents above relevance threshold."""
    return [doc for doc in documents if doc["relevance"] >= threshold]

def synthesize_answer(documents: List[dict], query: str) -> str:
    """Mock: Generate answer from selected documents."""
    doc_contents = [doc["content"] for doc in documents]
    return f"Based on {len(documents)} documents: {query} - {'; '.join(doc_contents[:2])}"

print("RAG tools defined")


## 6.3 RAG Reasoning Loop

The key insight: **Reasoning decides when to retrieve, when to select, and when to synthesize.**


In [ ]:
def rag_reasoning_step(state: RAGAgentState) -> RAGAgentState:
    """
    Execute one Read → Reason → Write cycle for RAG.
    
    Reasoning explicitly decides which phase to execute.
    """
    print(f"\n{'='*60}")
    print(f"RAG STEP: Phase = {state.current_phase}")
    print(f"{'='*60}\n")
    
    # ===== PHASE 1: READ from state =====
    print("📖 READ from state:")
    print(f"  Query: {state.query}")
    print(f"  Current phase: {state.current_phase}")
    print(f"  Retrieved docs: {len(state.retrieved_documents)}")
    print(f"  Selected docs: {len(state.selected_documents)}")
    
    # ===== PHASE 2: REASON about what to do =====
    print("\n🧠 REASON about next action:")
    
    thought = None
    action = None
    observation = None
    
    if state.current_phase == "analyze_query":
        thought = "I need to retrieve documents for this query"
        action = "retrieve"
        print(f"  Thought: {thought}")
        print(f"  Decision: {action}")
        
        # Execute retrieval
        docs = retrieve_documents(state.query, state.max_documents)
        state.retrieved_documents = docs
        observation = f"Retrieved {len(docs)} documents"
        state.current_phase = "select"
        
    elif state.current_phase == "select":
        thought = "I should select only the most relevant documents"
        action = "select"
        print(f"  Thought: {thought}")
        print(f"  Decision: {action}")
        
        # Execute selection
        selected = select_relevant_documents(state.retrieved_documents, threshold=0.85)
        state.selected_documents = selected
        observation = f"Selected {len(selected)} documents with relevance >= 0.85"
        state.current_phase = "synthesize"
        
    elif state.current_phase == "synthesize":
        thought = "I have selected documents, now I can generate an answer"
        action = "synthesize"
        print(f"  Thought: {thought}")
        print(f"  Decision: {action}")
        
        # Execute synthesis
        answer = synthesize_answer(state.selected_documents, state.query)
        state.final_answer = answer
        observation = f"Generated answer from {len(state.selected_documents)} documents"
        state.is_complete = True
        state.current_phase = "complete"
        
    else:
        thought = "Task is complete"
        action = "terminate"
        observation = "No more actions needed"
        state.is_complete = True
    
    print(f"  Observation: {observation}")
    
    # ===== PHASE 3: WRITE to state =====
    print("\n✍️  WRITE to state:")
    
    # Record reasoning decision
    state.reasoning_trace.append({
        "phase": state.current_phase,
        "thought": thought,
        "action": action,
        "observation": observation
    })
    print(f"  Recorded reasoning decision in trace")
    
    return state

print("RAG reasoning step defined")


## 6.4 Running the RAG Agent


In [ ]:
# Initialize RAG agent state
rag_state = RAGAgentState(
    query="What are transformers in machine learning?",
    max_documents=5
)

print("🚀 Running RAG Agent\n")

# Execute RAG loop
for iteration in range(5):
    rag_state = rag_reasoning_step(rag_state)
    
    print(f"\n📊 State after iteration {iteration + 1}:")
    print(json.dumps(rag_state.to_dict(), indent=2))
    
    if rag_state.is_complete:
        print(f"\n✅ RAG agent completed in {iteration + 1} iterations")
        break

print("\n" + "="*60)
print("🏁 Final RAG State:")
print("="*60)
print(f"\nQuery: {rag_state.query}")
print(f"\nRetrieved: {len(rag_state.retrieved_documents)} documents")
print(f"Selected: {len(rag_state.selected_documents)} documents")
print(f"\nFinal Answer:\n{rag_state.final_answer}")
print(f"\nReasoning Trace:")
for i, step in enumerate(rag_state.reasoning_trace, 1):
    print(f"  {i}. [{step['action']}] {step['thought']} → {step['observation']}")


## 6.5 RAG Validation: Retrieval Before Synthesis

Let's validate that the RAG agent follows the correct sequence:


In [ ]:
def validate_rag_sequence(state: RAGAgentState) -> dict:
    """
    Validate that RAG agent followed correct sequence:
    1. Retrieval must happen before selection
    2. Selection must happen before synthesis
    3. Reasoning must reference retrieved content
    """
    errors = []
    warnings = []
    
    # Check reasoning trace for correct sequence
    actions = [step["action"] for step in state.reasoning_trace]
    
    # Rule 1: Retrieval must come before selection
    if "select" in actions and "retrieve" not in actions:
        errors.append("Selection happened without retrieval")
    elif "select" in actions and "retrieve" in actions:
        retrieve_idx = actions.index("retrieve")
        select_idx = actions.index("select")
        if select_idx < retrieve_idx:
            errors.append("Selection happened before retrieval")
    
    # Rule 2: Selection must come before synthesis
    if "synthesize" in actions and "select" not in actions:
        errors.append("Synthesis happened without selection")
    elif "synthesize" in actions and "select" in actions:
        select_idx = actions.index("select")
        synth_idx = actions.index("synthesize")
        if synth_idx < select_idx:
            errors.append("Synthesis happened before selection")
    
    # Rule 3: If synthesis happened, must have selected documents
    if "synthesize" in actions and len(state.selected_documents) == 0:
        errors.append("Synthesis with no selected documents")
    
    # Rule 4: Final answer should only exist if synthesis happened
    if state.final_answer is not None and "synthesize" not in actions:
        errors.append("Final answer exists but synthesis never executed")
    
    # Rule 5: Check that reasoning references state
    for step in state.reasoning_trace:
        if step["action"] == "synthesize":
            if "documents" not in step["observation"].lower():
                warnings.append("Synthesis observation doesn't reference documents")
    
    return {
        "valid": len(errors) == 0,
        "errors": errors,
        "warnings": warnings,
        "sequence": actions
    }

# Validate the RAG execution
validation = validate_rag_sequence(rag_state)

print("🔍 RAG Sequence Validation:")
print(f"  Valid: {validation['valid']}")
print(f"  Action sequence: {' → '.join(validation['sequence'])}")

if validation["errors"]:
    print("\n  ❌ ERRORS:")
    for error in validation["errors"]:
        print(f"     - {error}")
else:
    print("\n  ✅ All sequence validations passed!")

if validation["warnings"]:
    print("\n  ⚠️  WARNINGS:")
    for warning in validation["warnings"]:
        print(f"     - {warning}")


## 6.6 Demonstration: Invalid RAG Sequence

Let's see what happens if we violate the sequence:


In [ ]:
# Create a BAD RAG state that violates sequence
bad_rag_state = RAGAgentState(
    query="What are transformers?",
    max_documents=5
)

# Manually create invalid sequence: synthesize WITHOUT retrieve
print("🚨 Creating INVALID RAG sequence (synthesize without retrieve)\n")

bad_rag_state.reasoning_trace.append({
    "phase": "synthesize",
    "thought": "I'll just synthesize without retrieving",
    "action": "synthesize",
    "observation": "Generated answer somehow"
})

bad_rag_state.final_answer = "Made up answer with no documents"
bad_rag_state.is_complete = True

# Validate
validation = validate_rag_sequence(bad_rag_state)

print("🔍 Validation of BROKEN RAG sequence:")
print(f"  Valid: {validation['valid']}")
print(f"  Action sequence: {' → '.join(validation['sequence'])}")

if validation["errors"]:
    print("\n  ❌ ERRORS DETECTED:")
    for error in validation["errors"]:
        print(f"     - {error}")
    print("\n  This is why validation is critical for RAG agents!")


## 6.7 Key Insights: RAG State-Reasoning Interaction

**What We Learned:**

1. **Reasoning controls the flow**: The agent decides when to retrieve, select, and synthesize
2. **State stores reasoning decisions**: Every decision is recorded in `reasoning_trace`
3. **Validation ensures correctness**: We can verify that retrieval happens before synthesis
4. **Reasoning references state**: Synthesis uses `selected_documents` from state

**Critical Pattern:**
```
READ state → REASON about phase → EXECUTE action → WRITE results → UPDATE phase
```

**Why This Matters:**
- ✅ **Transparent**: We can see exactly what the agent decided at each step
- ✅ **Verifiable**: We can validate the sequence is correct
- ✅ **Debuggable**: We can trace exactly where problems occur
- ✅ **Grounded**: Every decision is based on actual state, not assumptions

---

# Summary: Sections 4-6

| Section | Key Concept |
|---------|-------------|
| **Section 4** | Implemented Read → Reason → Write loop with observable state evolution |
| **Section 5** | Validated state-reasoning interactions; demonstrated broken vs corrected patterns |
| **Section 6** | Applied pattern to RAG agent; validated retrieval→selection→synthesis sequence |

---

## Next Sections Preview

In upcoming sections, we will:

- **Section 7:** Anti-patterns and common mistakes
- **Section 8:** Debugging state-reasoning interactions
- **Section 9:** Multi-agent state coordination
- **Section 10:** Production patterns and best practices

---

<a id="section-7"></a>
# Section 7: Visual Diagrams of State–Reasoning Interaction

Visual diagrams help us understand the flow of data and control between state and reasoning.

---

## 7.1 The Core Read → Reason → Write Loop

```
┌─────────────────────────────────────────────────────────────┐
│              THE FUNDAMENTAL LOOP                           │
└─────────────────────────────────────────────────────────────┘

    ┌─────────────────────────────────┐
    │      AGENT STATE                │
    │                                 │
    │  • query: "Find papers"         │
    │  • papers: []                   │
    │  • current_step: 1              │
    │  • is_complete: False           │
    └──────────────┬──────────────────┘
                   │
                   │ 1. READ
                   │ Extract relevant fields
                   │
                   ▼
    ┌──────────────────────────────────┐
    │   REASONING ENGINE               │
    │                                  │
    │  Inputs from state:              │
    │  • query = "Find papers"         │
    │  • papers = []                   │
    │  • current_step = 1              │
    │                                  │
    │  2. REASON                       │
    │  Decision logic:                 │
    │  if papers empty:                │
    │    → action = SEARCH             │
    │                                  │
    │  3. EXECUTE                      │
    │  search(query) → [10 papers]     │
    └──────────────┬───────────────────┘
                   │
                   │ 4. WRITE
                   │ Update state with results
                   │
                   ▼
    ┌─────────────────────────────────┐
    │   UPDATED STATE                 │
    │                                 │
    │  • query: "Find papers"         │
    │  • papers: [10 papers] ← NEW    │
    │  • current_step: 2 ← UPDATED    │
    │  • is_complete: False           │
    └──────────────┬──────────────────┘
                   │
                   │ Loop continues
                   │
                   └──────► (Back to READ)
```

**Key Observations:**
- State is passive storage
- Reasoning is active logic
- Write phase creates new state
- Loop repeats with enriched state

---

## 7.2 Separation of Concerns

```
┌──────────────────────────────────────────────────────────────┐
│           STATE vs REASONING: Clear Boundaries               │
└──────────────────────────────────────────────────────────────┘

STATE                              REASONING
(Data Container)                   (Decision Logic)
─────────────────                  ──────────────────

┌───────────────────┐             ┌─────────────────────┐
│ What I Know       │             │ What I Should Do    │
│                   │             │                     │
│ • User query      │────────────▶│ • Analyze context   │
│ • Retrieved data  │   READ      │ • Choose action     │
│ • Progress marker │             │ • Execute tool      │
│ • Results so far  │             │ • Generate output   │
└───────────────────┘             └─────────────────────┘
         ▲                                   │
         │                                   │
         │               WRITE               │
         └───────────────────────────────────┘

BOUNDARIES:
───────────
✅ State NEVER contains logic
✅ Reasoning NEVER stores data directly
✅ State is serializable
✅ Reasoning is procedural
✅ State survives across steps
✅ Reasoning is ephemeral (unless traced)

WRONG (Mixed):                    RIGHT (Separated):
─────────────                     ──────────────────
state = {                         state = {
  "query": "...",                   "query": "...",
  "action": "search",  ❌           "papers": [],
  "next_step": fn()    ❌           "reasoning_trace": []
}                                 }
                                  
                                  def reason(state):
                                    if not state["papers"]:
                                      return "search"
```

---

## 7.3 Interaction Boundaries

```
┌──────────────────────────────────────────────────────────────┐
│              READ-ONLY vs READ-WRITE BOUNDARIES              │
└──────────────────────────────────────────────────────────────┘

                    STATE FIELDS
    ┌───────────────────────────────────────┐
    │                                       │
    │  READ-ONLY (Inputs)                   │
    │  ┌─────────────────────────────────┐  │
    │  │ • initial_query                 │  │
    │  │ • max_results                   │  │
    │  │ • config_params                 │  │
    │  └─────────────────────────────────┘  │
    │           │                            │
    │           │ Reasoning can READ         │
    │           │ but NEVER WRITE            │
    │           ▼                            │
    │    ┌─────────────┐                     │
    │    │  REASONING  │                     │
    │    └─────────────┘                     │
    │           │                            │
    │           │ Reasoning can READ         │
    │           │ AND WRITE                  │
    │           ▼                            │
    │  READ-WRITE (Working Memory)          │
    │  ┌─────────────────────────────────┐  │
    │  │ • papers: []                    │  │
    │  │ • current_step: 1               │  │
    │  │ • reasoning_trace: []           │  │
    │  └─────────────────────────────────┘  │
    │           │                            │
    │           │ Reasoning ONLY WRITES      │
    │           │ (final output)             │
    │           ▼                            │
    │  WRITE-ONLY (Outputs)                 │
    │  ┌─────────────────────────────────┐  │
    │  │ • final_summary: None           │  │
    │  │ • is_complete: False            │  │
    │  └─────────────────────────────────┘  │
    │                                       │
    └───────────────────────────────────────┘

VALIDATION RULES:
────────────────
1. READ-ONLY fields must never change
2. READ-WRITE fields can grow (append) or update
3. WRITE-ONLY fields set once at completion
4. History fields (traces) are append-only
```

---

## 7.4 Single-Agent vs Multi-Agent Interaction

```
┌──────────────────────────────────────────────────────────────┐
│                   SINGLE AGENT                               │
└──────────────────────────────────────────────────────────────┘

        State ──► Reasoning ──► Updated State
          ▲                           │
          │                           │
          └───────────────────────────┘
              (Same agent loop)


┌──────────────────────────────────────────────────────────────┐
│                  MULTI-AGENT SYSTEM                          │
└──────────────────────────────────────────────────────────────┘

    AGENT 1                           AGENT 2
    ───────                           ───────
    
    State₁                            State₂
    ┌──────────┐                      ┌──────────┐
    │ query    │                      │ draft    │
    │ context  │                      │ critique │
    │ draft    │◄─────────────────────│ decision │
    └────┬─────┘                      └────▲─────┘
         │                                 │
         ▼                                 │
    Reasoning₁                        Reasoning₂
    ┌──────────┐                      ┌──────────┐
    │ Write    │                      │ Critique │
    │ draft    │──────────────────────►│ draft    │
    └──────────┘                      └──────────┘
         │                                 │
         │                                 │
         └─────► Output₁ ─────────────────┘
                 (becomes Input₂)

HANDOFF REQUIREMENTS:
────────────────────
1. Agent₁ writes explicit output
2. Agent₂ reads from well-defined input field
3. Reasoning artifacts are preserved
4. State contracts enforced at boundaries
```

---

## 7.5 Mermaid Diagram: State Flow

Below is a Mermaid diagram showing the state evolution through reasoning steps:


```mermaid
graph TD
    A[Initial State<br/>query, papers=[]] -->|READ| B[Reasoning Step 1]
    B -->|REASON: Need papers| C[Execute: search]
    C -->|WRITE| D[State 2<br/>papers=[10], step=2]
    
    D -->|READ| E[Reasoning Step 2]
    E -->|REASON: Filter papers| F[Execute: filter]
    F -->|WRITE| G[State 3<br/>filtered=[3], step=3]
    
    G -->|READ| H[Reasoning Step 3]
    H -->|REASON: Synthesize| I[Execute: summarize]
    I -->|WRITE| J[Final State<br/>summary=..., complete=True]
    
    style A fill:#e1f5ff
    style D fill:#e1f5ff
    style G fill:#e1f5ff
    style J fill:#c8e6c9
    style B fill:#fff9c4
    style E fill:#fff9c4
    style H fill:#fff9c4
```

**Diagram Explanation:**
- **Blue boxes**: State snapshots at different points
- **Yellow boxes**: Reasoning steps that read state and decide
- **Green box**: Final completed state
- **Arrows**: Flow of control (Read → Reason → Write)

Each reasoning step:
1. Reads current state
2. Decides what to do
3. Executes action
4. Writes results back to state

---

<a id="section-8"></a>
# Section 8: State ↔ Reasoning in Multi-Agent Systems (MAS)

In multi-agent systems, each agent has its own state and reasoning, but they must coordinate through **explicit handoffs**.

---

## 8.1 The Challenge: Agent Coordination

**Single Agent:**
- One state
- One reasoning loop
- Self-contained

**Multi-Agent:**
- Multiple states
- Multiple reasoning loops
- Must coordinate!

**The Problem:**
- How does Agent₂ know what Agent₁ decided?
- How does Agent₂'s state get the right inputs?
- How do we validate the handoff?

---

## 8.2 Multi-Agent State Schema

Let's build a Writer-Critic system:


In [ ]:
@dataclass
class WriterAgentState:
    """State for the Writer agent."""
    # READ-ONLY: Inputs
    topic: str
    
    # READ-WRITE: Working memory
    reasoning_trace: List[dict] = field(default_factory=list)
    
    # WRITE-ONLY: Outputs (becomes input to Critic)
    draft: Optional[str] = None
    is_complete: bool = False

@dataclass
class CriticAgentState:
    """State for the Critic agent."""
    # READ-ONLY: Input from Writer (contract!)
    draft_to_review: Optional[str] = None
    
    # READ-WRITE: Working memory
    reasoning_trace: List[dict] = field(default_factory=list)
    
    # WRITE-ONLY: Outputs (becomes input back to Writer)
    critique: Optional[str] = None
    approval: bool = False
    is_complete: bool = False

@dataclass
class SharedMASState:
    """Shared state for multi-agent coordination."""
    # Coordination fields
    current_agent: str = "writer"  # "writer" or "critic"
    iteration: int = 1
    max_iterations: int = 3
    
    # Agent-specific states
    writer_state: Optional[WriterAgentState] = None
    critic_state: Optional[CriticAgentState] = None
    
    # Final output
    final_approved_draft: Optional[str] = None
    is_complete: bool = False

print("Multi-agent state schemas defined")


## 8.3 Implementing Writer and Critic Reasoning

Each agent has its own reasoning function that operates on its own state:


In [ ]:
def writer_reasoning(state: WriterAgentState, critique: Optional[str] = None) -> WriterAgentState:
    """
    Writer agent reasoning: Generate or revise draft.
    
    READ: topic, critique (if exists)
    REASON: Should I write new draft or revise?
    WRITE: draft, reasoning_trace
    """
    print("\n📝 WRITER AGENT")
    print("-" * 40)
    
    # READ
    topic = state.topic
    print(f"READ: topic = '{topic}'")
    if critique:
        print(f"READ: critique = '{critique}'")
    
    # REASON
    if critique:
        thought = "I received critique, I should revise my draft"
        action = "revise"
        draft = f"REVISED DRAFT on {topic}: Incorporating feedback: {critique[:50]}..."
    else:
        thought = "No critique yet, I should write initial draft"
        action = "write"
        draft = f"INITIAL DRAFT on {topic}: This is a comprehensive discussion of {topic}."
    
    print(f"REASON: {thought}")
    print(f"ACTION: {action}")
    
    # WRITE
    state.draft = draft
    state.reasoning_trace.append({
        "thought": thought,
        "action": action,
        "output": "draft generated"
    })
    state.is_complete = True
    
    print(f"WRITE: draft generated ({len(draft)} chars)")
    print(f"WRITE: reasoning recorded")
    
    return state

def critic_reasoning(state: CriticAgentState) -> CriticAgentState:
    """
    Critic agent reasoning: Review draft.
    
    READ: draft_to_review
    REASON: Is it good enough?
    WRITE: critique, approval, reasoning_trace
    """
    print("\n🔍 CRITIC AGENT")
    print("-" * 40)
    
    # READ
    draft = state.draft_to_review
    print(f"READ: draft_to_review (length: {len(draft) if draft else 0} chars)")
    
    if not draft:
        # MISSING REASONING ARTIFACT!
        print("ERROR: No draft to review!")
        state.critique = "ERROR: No draft provided"
        state.approval = False
        state.is_complete = True
        return state
    
    # REASON (simple mock logic)
    if "REVISED" in draft:
        thought = "This draft has been revised, looks good"
        action = "approve"
        critique = "Good work incorporating feedback!"
        approval = True
    else:
        thought = "Initial draft needs improvement"
        action = "request_revision"
        critique = "Please add more detail about practical applications"
        approval = False
    
    print(f"REASON: {thought}")
    print(f"ACTION: {action}")
    
    # WRITE
    state.critique = critique
    state.approval = approval
    state.reasoning_trace.append({
        "thought": thought,
        "action": action,
        "output": critique
    })
    state.is_complete = True
    
    print(f"WRITE: critique = '{critique}'")
    print(f"WRITE: approval = {approval}")
    
    return state

print("Writer and Critic reasoning functions defined")


## 8.4 The Handoff: Reasoning Output → State Input

The critical pattern: **Agent₁'s reasoning output becomes Agent₂'s state input**


In [ ]:
# Initialize multi-agent system
mas_state = SharedMASState(
    writer_state=WriterAgentState(topic="Transformer Architectures"),
    critic_state=CriticAgentState()
)

print("🚀 Multi-Agent System Execution\n")
print("="*60)

# Iteration 1: Writer creates initial draft
print("\n【 ITERATION 1 】")
print("="*60)

# Writer reasons
mas_state.writer_state = writer_reasoning(mas_state.writer_state)

# HANDOFF: Writer's output → Critic's input
print("\n🔄 HANDOFF: Writer → Critic")
print("-" * 40)
draft_from_writer = mas_state.writer_state.draft
print(f"Transferring 'draft' from Writer's state to Critic's state")
print(f"Draft: '{draft_from_writer[:80]}...'")

# Set Critic's input
mas_state.critic_state.draft_to_review = draft_from_writer

# Critic reasons
mas_state.critic_state = critic_reasoning(mas_state.critic_state)

# Check if approved
if mas_state.critic_state.approval:
    mas_state.final_approved_draft = draft_from_writer
    mas_state.is_complete = True
    print("\n✅ Draft APPROVED! MAS complete.")
else:
    print("\n⚠️  Draft needs revision.")
    
# Iteration 2: Writer revises based on critique
if not mas_state.is_complete:
    print("\n【 ITERATION 2 】")
    print("="*60)
    
    # HANDOFF: Critic's output → Writer's input
    print("\n🔄 HANDOFF: Critic → Writer")
    print("-" * 40)
    critique_from_critic = mas_state.critic_state.critique
    print(f"Transferring 'critique' from Critic's state to Writer")
    print(f"Critique: '{critique_from_critic}'")
    
    # Writer revises
    mas_state.writer_state = writer_reasoning(mas_state.writer_state, critique_from_critic)
    
    # HANDOFF: Writer's revised draft → Critic's input
    print("\n🔄 HANDOFF: Writer → Critic (revised)")
    print("-" * 40)
    revised_draft = mas_state.writer_state.draft
    print(f"Transferring revised 'draft' from Writer to Critic")
    print(f"Draft: '{revised_draft[:80]}...'")
    
    # Reset critic state for new review
    mas_state.critic_state = CriticAgentState(draft_to_review=revised_draft)
    mas_state.critic_state = critic_reasoning(mas_state.critic_state)
    
    if mas_state.critic_state.approval:
        mas_state.final_approved_draft = revised_draft
        mas_state.is_complete = True
        print("\n✅ Revised draft APPROVED! MAS complete.")

print("\n" + "="*60)
print("🏁 Multi-Agent System Final State")
print("="*60)
print(f"\nFinal Draft: {mas_state.final_approved_draft}")
print(f"\nWriter Reasoning Trace: {len(mas_state.writer_state.reasoning_trace)} steps")
print(f"Critic Reasoning Trace: {len(mas_state.critic_state.reasoning_trace)} steps")


## 8.5 Validation: Checking for Missing Reasoning Artifacts


In [ ]:
def validate_mas_handoff(
    writer_state: WriterAgentState,
    critic_state: CriticAgentState
) -> dict:
    """
    Validate multi-agent handoff.
    
    Checks:
    1. Writer produced draft (reasoning artifact)
    2. Critic received draft (state input contract)
    3. Critic produced critique (reasoning artifact)
    4. Both agents have reasoning traces
    """
    errors = []
    warnings = []
    
    # Check 1: Writer must have produced draft
    if not writer_state.draft:
        errors.append("Writer did not produce draft (missing reasoning artifact)")
    
    # Check 2: Critic must have received draft
    if not critic_state.draft_to_review:
        errors.append("Critic did not receive draft (handoff contract violated)")
    
    # Check 3: If Critic received draft, it should match Writer's draft
    if writer_state.draft and critic_state.draft_to_review:
        if writer_state.draft != critic_state.draft_to_review:
            warnings.append("Draft mismatch between Writer output and Critic input")
    
    # Check 4: Critic must have produced critique
    if not critic_state.critique:
        errors.append("Critic did not produce critique (missing reasoning artifact)")
    
    # Check 5: Both must have reasoning traces
    if len(writer_state.reasoning_trace) == 0:
        errors.append("Writer has no reasoning trace")
    
    if len(critic_state.reasoning_trace) == 0:
        errors.append("Critic has no reasoning trace")
    
    return {
        "valid": len(errors) == 0,
        "errors": errors,
        "warnings": warnings
    }

# Test validation on our execution
validation = validate_mas_handoff(mas_state.writer_state, mas_state.critic_state)

print("🔍 Multi-Agent Handoff Validation:")
print(f"  Valid: {validation['valid']}")

if validation["valid"]:
    print("\n  ✅ All handoff validations passed!")
    print(f"     - Writer produced draft: ✓")
    print(f"     - Critic received draft: ✓")
    print(f"     - Critic produced critique: ✓")
    print(f"     - Both have reasoning traces: ✓")
else:
    print("\n  ❌ ERRORS:")
    for error in validation["errors"]:
        print(f"     - {error}")

if validation["warnings"]:
    print("\n  ⚠️  WARNINGS:")
    for warning in validation["warnings"]:
        print(f"     - {warning}")


## 8.6 Demonstration: BROKEN Multi-Agent Handoff

Let's see what happens when the handoff contract is violated:


In [ ]:
print("🚨 Testing BROKEN Multi-Agent Handoff\n")

# Create states
broken_writer = WriterAgentState(topic="AI Safety")
broken_critic = CriticAgentState()

# Writer produces draft
broken_writer = writer_reasoning(broken_writer)

# VIOLATION: Forget to handoff draft to critic!
# (We skip: broken_critic.draft_to_review = broken_writer.draft)

print("\n❌ SKIPPED HANDOFF: Draft not transferred to Critic!")

# Critic tries to reason without draft
broken_critic = critic_reasoning(broken_critic)

# Validate
validation = validate_mas_handoff(broken_writer, broken_critic)

print("\n🔍 Validation of BROKEN handoff:")
print(f"  Valid: {validation['valid']}")

if not validation["valid"]:
    print("\n  ❌ ERRORS DETECTED:")
    for error in validation["errors"]:
        print(f"     - {error}")
    print("\n  This is why contracts and validation are critical for MAS!")


## 8.7 Why Contracts Are Required

**The Contract:**
```
Writer Output Schema:
  - draft: str (required)
  - reasoning_trace: List[dict]

Critic Input Schema:
  - draft_to_review: str (required)
```

**Without this contract:**
- Critic doesn't know what fields to expect
- Writer doesn't know what to provide
- Handoff can silently fail
- System breaks unpredictably

**With this contract:**
- ✅ Explicit interface
- ✅ Validatable
- ✅ Self-documenting
- ✅ Catches errors early

---

<a id="section-9"></a>
# Section 9: Common Interaction Failures

Let's examine the most common ways state-reasoning interactions fail.

---

## 9.1 Failure Mode 1: Reasoning Hidden in Prompts

**The Problem:** Putting reasoning logic in prompts instead of code.


In [ ]:
def failure_1_prompt_based_reasoning():
    """
    FAILURE MODE 1: Reasoning hidden in prompts.
    
    WHY IT FAILS:
    - No explicit state reads
    - No verifiable reasoning steps
    - Can hallucinate steps that didn't happen
    - No state writes
    """
    print("❌ FAILURE MODE 1: Prompt-Based Reasoning\n")
    
    state = {"query": "Find papers", "papers": []}
    
    # BAD: Put reasoning in prompt
    prompt = f"""
    You are an agent. Your task is: {state['query']}
    
    Think step by step:
    1. Search for papers
    2. Filter by relevance
    3. Summarize findings
    
    Now execute these steps and give me the result.
    """
    
    # Simulate LLM response (would be actual LLM call)
    llm_response = "I searched and found papers X, Y, Z. Summary: ..."
    
    print(f"Prompt sent to LLM:\n{prompt}\n")
    print(f"LLM Response: {llm_response}\n")
    
    # PROBLEMS:
    print("🚨 PROBLEMS:")
    print("  1. State wasn't actually read from structured data")
    print("  2. No way to verify steps actually happened")
    print("  3. LLM could claim to have done steps without doing them")
    print("  4. State wasn't updated - papers not in state!")
    print("  5. No reasoning trace to debug")
    
    print(f"\nFinal state: {state}")
    print("  → State is unchanged! Reasoning was invisible.")
    
    return state

failure_1_prompt_based_reasoning()


**Why This Fails:**
- Reasoning is in the **prompt**, not in **code**
- State is not explicitly read or written
- No verification that actions actually happened
- LLM can hallucinate steps
- No audit trail

**The Fix:** Use explicit Read → Reason → Write loop (as shown in Section 4)

---

## 9.2 Failure Mode 2: State Overwritten Incorrectly


In [ ]:
def failure_2_state_overwritten():
    """
    FAILURE MODE 2: Read-only state overwritten.
    
    WHY IT FAILS:
    - Original inputs lost
    - Can't replay from original state
    - Debugging becomes impossible
    - Violates architectural contracts
    """
    print("❌ FAILURE MODE 2: State Overwritten Incorrectly\n")
    
    state = {
        "original_query": "Find papers on transformers",  # Should be read-only!
        "papers": []
    }
    
    print(f"Initial state: {state}")
    
    # BAD: Overwriting read-only field
    state["original_query"] = "modified query"  # WRONG!
    
    # BAD: Replacing history instead of appending
    state["reasoning_trace"] = [{"step": 3}]  # Lost steps 1 and 2!
    
    print(f"\nAfter bad modifications: {state}")
    
    # PROBLEMS:
    print("\n🚨 PROBLEMS:")
    print("  1. original_query changed - can't trace back to user intent")
    print("  2. reasoning_trace replaced - history lost")
    print("  3. Can't replay execution")
    print("  4. Debugging is impossible")
    print("  5. Violates read-only contract")

failure_2_state_overwritten()


**Why This Fails:**
- **Read-only fields modified**: Original user intent lost
- **History replaced**: Previous steps erased
- **No replay possible**: Can't reconstruct what happened
- **Contract violated**: Breaks architectural assumptions

**The Fix:** 
- Use frozen dataclasses for read-only fields
- Always `.append()` to trace lists, never replace
- Validate state transitions (as shown in Section 5)

---

## 9.3 Failure Mode 3: Reasoning Steps Skipped


In [ ]:
def failure_3_steps_skipped():
    """
    FAILURE MODE 3: Required reasoning steps skipped.
    
    WHY IT FAILS:
    - Dependencies not met
    - Outputs generated without proper inputs
    - Results are ungrounded
    """
    print("❌ FAILURE MODE 3: Reasoning Steps Skipped\n")
    
    state = {
        "query": "Find papers on transformers",
        "papers": [],             # Should be filled by search
        "filtered_papers": [],    # Should be filled by filter
        "summary": None           # Should be generated last
    }
    
    print(f"Initial state: {state}")
    
    # BAD: Skip search and filter, go straight to summary
    print("\n⚠️  Skipping search step...")
    print("⚠️  Skipping filter step...")
    print("⚠️  Going straight to synthesis...")
    
    # Generate summary without papers
    state["summary"] = "Summary: Transformers are neural network architectures..."
    
    print(f"\nFinal state: {state}")
    
    # PROBLEMS:
    print("\n🚨 PROBLEMS:")
    print(f"  1. papers is empty: {state['papers']}")
    print(f"  2. filtered_papers is empty: {state['filtered_papers']}")
    print(f"  3. Summary exists but is UNGROUNDED!")
    print("  4. No evidence for claims in summary")
    print("  5. Validation would catch: synthesis without retrieval")

failure_3_steps_skipped()


**Why This Fails:**
- **Dependencies skipped**: Summary needs papers, but none retrieved
- **Ungrounded output**: Claims not based on evidence
- **Validation would catch**: Required sequence not followed
- **Hallucination risk**: Agent makes up information

**The Fix:**
- Validate sequence (as in Section 6: RAG validation)
- Check preconditions before each phase
- Require evidence before conclusions

---

## 9.4 Failure Mode 4: Reasoning Over Stale State


In [ ]:
def failure_4_stale_state():
    """
    FAILURE MODE 4: Reasoning over stale state.
    
    WHY IT FAILS:
    - Agent uses outdated information
    - Decisions don't reflect current reality
    - Common in multi-agent or async systems
    """
    print("❌ FAILURE MODE 4: Reasoning Over Stale State\n")
    
    # Step 1: Agent reads state
    state = {
        "papers": [
            {"id": 1, "title": "Paper A"},
            {"id": 2, "title": "Paper B"}
        ],
        "current_step": 1
    }
    
    print(f"Step 1: Agent reads state")
    print(f"  papers = {[p['title'] for p in state['papers']]}")
    
    # Simulate: Another agent/process updates state
    state["papers"].append({"id": 3, "title": "Paper C"})
    state["papers"].append({"id": 4, "title": "Paper D"})
    print(f"\n⚡ External update: 2 more papers added!")
    print(f"  Current papers = {[p['title'] for p in state['papers']]}")
    
    # Step 2: Agent reasons over STALE snapshot
    stale_papers = [{"id": 1, "title": "Paper A"}, {"id": 2, "title": "Paper B"}]  # Old copy
    
    print(f"\n⚠️  Agent reasoning over STALE state:")
    print(f"  Agent thinks papers = {[p['title'] for p in stale_papers]}")
    print(f"  Agent decides: 'Only 2 papers, search for more'")
    
    # PROBLEMS:
    print("\n🚨 PROBLEMS:")
    print("  1. Agent made decision based on stale data")
    print("  2. Already has 4 papers, but thinks it has 2")
    print("  3. Will do unnecessary search")
    print("  4. State and reasoning are out of sync")

failure_4_stale_state()


**Why This Fails:**
- **Stale read**: Agent works with outdated snapshot
- **Bad decisions**: Based on old information
- **Common in MAS**: Multiple agents updating shared state
- **Race conditions**: State changes between read and write

**The Fix:**
- Always read fresh state at start of each reasoning step
- Use locking/transactions in multi-agent systems
- Validate state freshness before major decisions
- Consider optimistic concurrency control

---

## 9.5 Summary of Failure Modes


| Failure Mode | Root Cause | Detection | Prevention |
|--------------|------------|-----------|------------|
| **Prompt-based reasoning** | Logic in prompts, not code | State unchanged after "reasoning" | Use explicit Read→Reason→Write |
| **State overwritten** | Read-only fields modified | Validation checks | Use frozen fields, append-only |
| **Steps skipped** | Dependencies not enforced | Sequence validation | Check preconditions |
| **Stale state** | Reading old snapshots | State version mismatch | Read fresh at each step |

---

<a id="section-10"></a>
# Section 10: Final Mental Model

Let's synthesize everything into a unified mental model.

---

## 10.1 The Unified Mental Model

```
┌──────────────────────────────────────────────────────────────┐
│       AGENT = STATE + REASONING (in feedback loop)          │
└──────────────────────────────────────────────────────────────┘

                    ┌─────────────────────┐
                    │  USER REQUEST       │
                    │  "Find papers..."   │
                    └──────────┬──────────┘
                               │
                               ▼
    ┌──────────────────────────────────────────────────┐
    │              AGENT STATE                          │
    │                                                   │
    │  ┌───────────────┐  ┌───────────────┐            │
    │  │ READ-ONLY     │  │ READ-WRITE    │            │
    │  │ • query       │  │ • papers      │            │
    │  │ • config      │  │ • trace       │            │
    │  └───────────────┘  └───────────────┘            │
    │  ┌───────────────┐                               │
    │  │ WRITE-ONLY    │                               │
    │  │ • output      │                               │
    │  │ • complete    │                               │
    │  └───────────────┘                               │
    └───────────────────────┬──────────────────────────┘
                            │
         ┌──────────────────┴──────────────────┐
         │                                     │
         ▼                                     │
    ┌─────────────┐                            │
    │  1. READ    │  Extract relevant          │
    │             │  fields from state         │
    └──────┬──────┘                            │
           │                                   │
           ▼                                   │
    ┌─────────────┐                            │
    │  2. REASON  │  Decide next action        │
    │             │  based on state            │
    └──────┬──────┘                            │
           │                                   │
           ▼                                   │
    ┌─────────────┐                            │
    │  3. EXECUTE │  Call tools, APIs,         │
    │             │  external services         │
    └──────┬──────┘                            │
           │                                   │
           ▼                                   │
    ┌─────────────┐                            │
    │  4. WRITE   │  Update state with         │
    │             │  results + trace           │
    └──────┬──────┘                            │
           │                                   │
           └───────────────────────────────────┘
                      (loop until complete)
                            │
                            ▼
    ┌──────────────────────────────────────────────────┐
    │                  FINAL OUTPUT                     │
    │         Structured, verifiable, traceable         │
    └──────────────────────────────────────────────────┘


KEY INVARIANTS:
───────────────
1. State is PASSIVE (just data)
2. Reasoning is ACTIVE (decision logic)
3. Read-only fields NEVER change
4. Every decision is TRACED
5. Every step READS fresh state
6. Every step WRITES results back
7. Multi-agent uses CONTRACTS
```

---

## 10.2 Reusable Interaction Checklist

Use this checklist when designing any agent:

### ☑️ State Design Checklist

```
□ Define READ-ONLY fields (inputs, config)
□ Define READ-WRITE fields (working memory)
□ Define WRITE-ONLY fields (outputs, flags)
□ Document which fields are append-only
□ Use typed schema (dataclass, TypedDict)
□ Include reasoning_trace field
□ Include step counter and completion flag
```

### ☑️ Reasoning Step Checklist

```
□ Always READ from current state first
□ Log/record the reasoning decision
□ Execute action based on decision
□ WRITE results back to state
□ APPEND to trace, never replace
□ Increment step counter
□ Check termination condition
```

### ☑️ Validation Checklist

```
□ Verify read-only fields unchanged
□ Verify trace grew by exactly 1
□ Verify step counter incremented
□ Validate action sequence (if ordered)
□ Check for missing artifacts
□ Ensure outputs are grounded in inputs
```

### ☑️ Multi-Agent Checklist

```
□ Define output schema for each agent
□ Define input schema for each agent
□ Document handoff contracts
□ Validate handoff before proceeding
□ Handle missing artifacts gracefully
□ Track per-agent reasoning traces
```

---

## 10.3 Quick Reference: Do's and Don'ts

### ✅ DO:

| Pattern | Example |
|---------|---------|
| Separate state from reasoning | `state = {...}`, `def reason(state): ...` |
| Use typed schemas | `@dataclass` or `TypedDict` |
| Read fresh state each step | `current = read_state()` |
| Append to traces | `state.trace.append(...)` |
| Validate transitions | `validate(before, after)` |
| Record every decision | `trace.append({thought, action, obs})` |
| Use contracts for MAS | Define input/output schemas |

### ❌ DON'T:

| Anti-Pattern | Why It Fails |
|--------------|--------------|
| Put reasoning in prompts | Can't verify, can hallucinate |
| Modify read-only fields | Loses original context |
| Replace trace history | Loses audit trail |
| Skip reasoning steps | Outputs ungrounded |
| Read stale state | Bad decisions |
| Mix state and logic | Hard to debug/test |

---

## 10.4 Scaling in Production


In [ ]:
# Production Patterns Summary

production_patterns = """
┌────────────────────────────────────────────────────────────────┐
│             PRODUCTION SCALING PATTERNS                         │
└────────────────────────────────────────────────────────────────┘

1. STATE PERSISTENCE
   ─────────────────
   • Serialize state to JSON/DB after each step
   • Enable pause/resume at any point
   • Support replay for debugging
   
   Pattern:
   for step in steps:
       state = load_state(state_id)
       state = reason(state)
       save_state(state_id, state)

2. OBSERVABILITY
   ─────────────
   • Log every read/reason/write
   • Emit metrics (step count, latency, errors)
   • Create dashboards for state evolution
   
   Pattern:
   def instrumented_step(state):
       log.info(f"READ: {state.current_step}")
       result = reason(state)
       log.info(f"WRITE: {result.action}")
       metrics.emit("step_completed", state.current_step)
       return result

3. ERROR RECOVERY
   ──────────────
   • Catch errors, write to state
   • Support retry from last good state
   • Implement circuit breakers
   
   Pattern:
   try:
       state = reason(state)
   except Exception as e:
       state.error = str(e)
       state.is_complete = True  # Fail gracefully
       save_state(state_id, state)

4. MULTI-AGENT COORDINATION
   ────────────────────────
   • Use message queues for handoffs
   • Implement idempotent handlers
   • Version state schemas
   
   Pattern:
   writer_output = await queue.receive("writer_complete")
   critic_input = transform(writer_output, CriticInputSchema)
   await queue.send("critic_input", critic_input)

5. TESTING
   ───────
   • Unit test reasoning functions
   • Integration test state transitions
   • Property-based testing for invariants
   
   Pattern:
   def test_reasoning_step():
       before = create_test_state()
       after = reason(before)
       assert after.step == before.step + 1
       assert before.initial_query == after.initial_query  # Read-only
       assert len(after.trace) == len(before.trace) + 1

6. VERSIONING
   ──────────
   • Version state schemas
   • Migrate states between versions
   • Support backward compatibility
   
   Pattern:
   @dataclass
   class AgentStateV2:
       schema_version: str = "2.0"
       # New fields with defaults for migration
       new_field: Optional[str] = None
"""

print(production_patterns)


## 10.5 Final Summary

### The Three Laws of State-Reasoning Interaction

1. **State is passive, reasoning is active**
   - State stores data; reasoning makes decisions
   - Never mix them

2. **Every step follows Read → Reason → Write**
   - Read current state
   - Decide and execute action
   - Write results back
   - Repeat

3. **Respect field boundaries**
   - Read-only: never modify
   - Read-write: can evolve
   - Write-only: set at end
   - Trace: append-only

### What You Learned

| Section | Key Concept |
|---------|-------------|
| **1** | State and reasoning are separate but connected |
| **2** | Read → Reason → Write is the fundamental loop |
| **3** | State fields have clear access boundaries |
| **4** | Implement the loop with observable state |
| **5** | Validate every state transition |
| **6** | RAG agents follow retrieve → select → synthesize |
| **7** | Visual diagrams clarify the architecture |
| **8** | Multi-agent systems need contracts |
| **9** | Common failures: prompts, overwrites, skips, stale |
| **10** | Unified mental model and production patterns |

---

## 🎯 Key Takeaway

> **Agents are state machines where reasoning transforms state through explicit Read → Reason → Write loops. By keeping state and reasoning separate, respecting field boundaries, and validating transitions, you build agents that are transparent, debuggable, and production-ready.**

---

# End of Notebook

You are now equipped to design and implement robust state-reasoning interactions in any agent system.

**Next Steps:**
1. Apply these patterns to your own agents
2. Build validation into your development workflow
3. Use contracts for multi-agent coordination
4. Instrument with observability for production
